<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-30 14:28:49
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.35 C
-------------------
Today PnL: 57.23 K (0.43%)
Current PnL: -14.67 L (-10.22%)
CY Booked + Current PnL: -7.93 L (-5.53%)
-------------------
Total profit:  3.92 L
Total loss:  -18.59 L
-------------------
Total Booked + Current PnL: 19.05 L (16.35%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (5.0%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.07 C
Est FTT PnL: 71.99 L (53.38%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 9.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,45.16,102730.0,24368.0,9667.0,0.47,31.10,9.41,43.43,77.0,2.52,0.76,38.07,XY24,NTT,MISC
84,UJJIVANSFB,52.77,53.00,52.86,M-SC,94.62,133002.0,-9477.0,10095.0,0.96,-6.65,7.59,0.44,243.0,-0.94,0.99,56.98,OX40N,NTT,BANKS
85,UNIONBANK,123.87,163.00,0.15,M-LC,60.22,174359.0,33519.0,10967.0,2.88,23.80,6.29,31.59,89.0,3.06,1.29,51.46,XY24,NTT,BANKS
36,HINDALCO,651.95,761.55,-19.90,H-LC,86.02,110720.0,6408.0,11127.0,-0.77,6.14,10.05,16.81,27.0,0.58,0.82,23.18,X5K,ATH,METALS
68,SBIN,760.30,863.00,-10.56,M-LC,41.94,217035.0,15556.0,11655.0,1.69,7.72,5.37,13.51,76.0,1.33,1.61,21.29,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2804.19,3545.00,-33.15,H-SC,38.71,153747.0,-483.0,41235.0,0.96,-0.31,26.82,26.42,125.0,-0.01,1.14,21.24,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-12.32,H-SC,76.34,164440.0,-230.0,69311.0,1.26,-0.14,42.15,41.95,121.0,-0.00,1.22,7.32,X40N,ATH,AC


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-15.90,H-SC,84.95,166721.0,-29253.0,102067.0,8.05,-14.93,61.22,37.16,130.0,-0.29,1.24,24.51,XY24,NTT,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,100.00,101299.0,336.0,94198.0,4.15,0.33,92.99,93.63,64.0,0.00,0.75,50.88,SR,ATH,CHEMICALS
51,KPIGREEN,497.21,732.16,26.30,H-SC,64.52,129137.0,3840.0,55361.0,3.50,3.06,42.87,47.25,139.0,0.07,0.96,61.56,MH,ATH,POWER
85,UNIONBANK,123.87,163.00,0.15,M-LC,60.22,174359.0,33519.0,10967.0,2.88,23.80,6.29,31.59,89.0,3.06,1.29,51.46,XY24,NTT,BANKS
27,DIXON,14367.00,18940.15,-35.35,H-MC,43.01,119032.0,4096.0,32484.0,2.81,3.56,27.29,31.83,99.0,0.13,0.88,36.43,X40N,ATH,IT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,6039.03,-7.62,H-MC,21.51,148596.0,-2317.0,62767.0,-1.73,-1.54,42.24,40.06,102.0,-0.04,1.10,21.54,AR,ATH,AUTO
69,SFL,1016.35,1287.00,41.82,H-SC,96.77,187153.0,-75065.0,144894.0,-1.56,-28.63,77.42,26.63,144.0,-0.52,1.39,16.36,XY24,NTT,MISC
87,VALIANTORG,512.64,838.00,-353.90,H-SC,4.30,107900.0,-25386.0,109982.0,-1.51,-19.05,101.93,63.47,148.0,-0.23,0.80,80.87,XR,NTT,CHEMICALS
54,LTIM,5564.16,7230.20,-1.02,H-LC,62.37,190548.0,-9762.0,69741.0,-1.49,-4.87,36.60,29.94,38.0,-0.14,1.41,31.64,X200,ATH,IT
55,MASFIN,326.60,402.39,-19.15,H-SC,58.06,92100.0,-5880.0,28615.0,-1.40,-6.00,31.07,23.21,137.0,-0.21,0.68,33.58,XR,ATH,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,52.86,M-SC,94.62,133002.0,-9477.0,10095.0,0.96,-6.65,7.59,0.44,243.0,-0.94,0.99,56.98,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.80,X-MC,24.73,88403.0,-14249.0,14242.0,-0.48,-13.88,16.11,-0.00,58.0,-1.00,0.66,34.38,OX40N,NTT,IT
50,KANSAINER,299.63,340.0,-67.47,H-SC,30.11,227070.0,-42597.0,78930.0,1.69,-15.80,34.76,13.47,142.0,-0.54,1.68,14.25,XY24,NTT,PAINTS
89,VIPIND,488.80,489.0,-211.21,H-SC,93.55,80242.0,-15074.0,15110.0,-1.37,-15.81,18.83,0.04,154.0,-1.00,0.60,58.27,OX40N,NTT,MISC
43,INDIGOPNTS,1407.73,1408.0,119.87,H-SC,77.42,144249.0,-30310.0,30336.0,0.19,-17.36,21.03,0.02,127.0,-1.00,1.07,25.09,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.10,X-LC,40.86,269545.0,-26389.0,120298.0,0.42,-8.92,44.63,31.73,1.0,-0.22,2.00,7.89,X40,BTT,IT
45,INFY,1461.46,2275.00,-10.27,X-LC,49.46,287295.0,25694.0,119917.0,-0.19,9.82,41.74,55.67,2.0,0.21,2.13,16.53,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-18.00,X-LC,22.58,256861.0,-13486.0,48110.0,-0.59,-4.99,18.73,12.81,6.0,-0.28,1.91,7.42,XY25,NTT,FMCG
58,NESTLEIND,2268.60,2755.00,-8.92,X-LC,52.69,288698.0,23272.0,33633.0,0.36,8.77,11.65,21.44,9.0,0.69,2.14,15.51,XY25,NTT,FMCG
16,BRITANNIA,4983.33,6446.05,17.22,X-LC,69.89,268939.0,39706.0,27566.0,0.90,17.32,10.25,29.35,10.0,1.44,1.99,27.79,XY25,ATH,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UNIONBANK,123.87,163.0,0.15,M-LC,60.22,174359.0,33519.0,10967.0,2.88,23.8,6.29,31.59,89.0,3.06,1.29,51.46,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,45.16,102730.0,24368.0,9667.0,0.47,31.1,9.41,43.43,77.0,2.52,0.76,38.07,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,89.46,M-MC,48.39,227562.0,2600.0,165051.0,0.34,1.16,72.53,74.53,185.0,0.02,1.69,32.12,XY24,BTT,STEEL
51,KPIGREEN,497.21,732.16,26.30,H-SC,64.52,129137.0,3840.0,55361.0,3.50,3.06,42.87,47.25,139.0,0.07,0.96,61.56,MH,ATH,POWER
19,CAMS,3643.00,5250.99,7.21,H-SC,80.65,120358.0,18354.0,26671.0,0.13,17.99,22.16,44.14,124.0,0.69,0.89,37.16,X40N,ATH,MISC
48,JIOFIN,310.58,387.00,6.45,M-LC,92.47,228200.0,10794.0,42696.0,0.79,4.96,18.71,24.61,95.0,0.25,1.69,62.25,XY24,BTT,FINANCE
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,45.16,102730.0,24368.0,9667.0,0.47,31.10,9.41,43.43,77.0,2.52,0.76,38.07,XY24,NTT,MISC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-37.55,M-SC,91.40,205800.0,34179.0,127884.0,-0.20,19.92,62.14,94.43,198.0,0.27,1.53,51.89,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-3.77,M-LC,72.04,164684.0,13739.0,95714.0,0.21,9.10,58.12,72.51,69.0,0.14,1.22,13.44,XR,NTT,IT
42,INDIAMART,2327.09,4871.06,-47.52,H-SC,88.17,136629.0,13293.0,121531.0,-1.17,10.78,88.95,109.32,122.0,0.11,1.01,34.98,AR,ATH,MISC
51,KPIGREEN,497.21,732.16,26.30,H-SC,64.52,129137.0,3840.0,55361.0,3.50,3.06,42.87,47.25,139.0,0.07,0.96,61.56,MH,ATH,POWER
57,NATIONALUM,189.63,247.44,-49.40,H-MC,75.27,102120.0,1616.0,29023.0,0.97,1.61,28.42,30.49,97.0,0.06,0.76,34.53,MH,ATH,METALS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,97.85,71962.0,-41587.0,81641.0,1.90,-36.62,113.45,35.27,261.0,-0.51,0.53,76.13,XR,NTT,HOTELS
55,MASFIN,326.60,402.39,-19.15,H-SC,58.06,92100.0,-5880.0,28615.0,-1.40,-6.00,31.07,23.21,137.0,-0.21,0.68,33.58,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,100.00,101299.0,336.0,94198.0,4.15,0.33,92.99,93.63,64.0,0.00,0.75,50.88,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,45.16,102730.0,24368.0,9667.0,0.47,31.10,9.41,43.43,77.0,2.52,0.76,38.07,XY24,NTT,MISC
87,VALIANTORG,512.64,838.00,-353.90,H-SC,4.30,107900.0,-25386.0,109982.0,-1.51,-19.05,101.93,63.47,148.0,-0.23,0.80,80.87,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,100.00,101299.0,336.0,94198.0,4.15,0.33,92.99,93.63,64.0,0.00,0.75,50.88,SR,ATH,CHEMICALS
49,JPPOWER,18.73,26.20,-30.97,L-SC,98.92,139116.0,-2726.0,59291.0,1.49,-1.92,42.62,39.88,259.0,-0.05,1.03,40.98,XY24,NTT,POWER
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,97.85,71962.0,-41587.0,81641.0,1.90,-36.62,113.45,35.27,261.0,-0.51,0.53,76.13,XR,NTT,HOTELS
69,SFL,1016.35,1287.00,41.82,H-SC,96.77,187153.0,-75065.0,144894.0,-1.56,-28.63,77.42,26.63,144.0,-0.52,1.39,16.36,XY24,NTT,MISC
66,SAMMAANCAP,170.35,326.00,-175.00,M-SC,95.70,85800.0,-16410.0,109798.0,1.64,-16.06,127.97,91.37,206.0,-0.15,0.64,39.28,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.16
1,25,42.42
2,50,71.06


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.89
LC    34.44
MC    21.63
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.36
X40      12.34
XY25     12.22
X40N     11.85
XR       10.27
OX40N     8.23
AR        7.82
MH        1.72
X5K       1.49
X200      1.41
SR        1.25
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.83
M-SC    16.17
H-LC    12.60
X-LC    12.35
H-MC     8.66
M-LC     8.42
X-MC     7.62
M-MC     4.98
X-SC     4.40
L-SC     2.49
L-LC     1.07
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.32,-1.33,35.20
IT,11.89,-10.64,65.58
BANKS,8.06,-4.91,47.71
FINANCE,8.04,-11.16,51.40
MISC,6.58,-21.97,69.67
PAINTS,5.70,-13.58,35.96
ELECTRICAL,5.03,-3.50,48.32
HEALTHCARE,4.52,-5.73,36.61
AUTO,4.37,-14.61,65.00


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2701017.0,25
XR,1073684.0,13
AR,1015253.0,9
X40,656010.0,9
X40N,563756.0,11
XY25,471441.0,8
OX40N,321887.0,11
SR,175059.0,2
MH,84384.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1974580.0,20
M-SC,1763308.0,21
H-LC,570518.0,11
X-LC,509198.0,7
M-MC,496787.0,4
H-MC,467740.0,7
X-MC,425554.0,6
X-SC,324241.0,5
M-LC,307938.0,6


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,850607.0,6
M-SC,XY24,815045.0,7
H-SC,AR,521516.0,3
M-MC,XY24,386816.0,3
M-SC,XR,331676.0,4
X-LC,X40,240215.0,2
H-SC,X40N,233183.0,4
H-MC,XY24,223231.0,2
H-SC,XR,217811.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
